## Lottery Tickets Discovery

Find boosting tickets for Deep Hedging:
1. LR exploration
2. Sparsity ablation
3. Characterization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from copy import deepcopy

from src.utils.config import load_config, get_device
from src.models.deep_hedging import DeepHedgingNetwork
from src.models.trainer import Trainer
from src.data.preprocessor import create_dataloaders
from src.pruning.magnitude import magnitude_pruning, rewind_weights, get_sparsity  
from src.evaluation.metrics import compute_all_metrics
from src.utils.visualization import plot_convergence_comparison, plot_sparsity_performance

### Setup

In [2]:
config = load_config('../config.yaml')
device = get_device(config)

# Load data
S_train = np.load('../data/processed/S_train.npy')
v_train = np.load('../data/processed/v_train.npy')
Z_train = np.load('../data/processed/Z_train.npy')

S_val = np.load('../data/processed/S_val.npy')
v_val = np.load('../data/processed/v_val.npy')
Z_val = np.load('../data/processed/Z_val.npy')

S_test = np.load('../data/processed/S_test.npy')
v_test = np.load('../data/processed/v_test.npy')
Z_test = np.load('../data/processed/Z_test.npy')

batch_size = config['training']['batch_size'] or 256
train_loader, val_loader, test_loader = create_dataloaders(
    S_train, v_train, Z_train, S_val, v_val, Z_val, S_test, v_test, Z_test,
    batch_size, config['compute']['num_parallel_workers']
)

K = config['data']['heston']['K']
T = config['data']['T']
dt = config['data']['dt']

# Load baseline metrics
with open('../experiments/baseline/metrics.json', 'r') as f:
    baseline_metrics = json.load(f)

baseline_cvar = baseline_metrics['cvar_005']
print(f"Baseline CVaR: {baseline_cvar:.6f}")

🔍 Auto-detected device: cpu
 Using device: cpu


FileNotFoundError: [Errno 2] No such file or directory: '../experiments/baseline/metrics.json'

### Experiment 2.1: Learning Rate Exploration

In [ ]:
LR_candidates = config['pruning']['pruning_lr_candidates']
results_lr = {}

for lr in LR_candidates:
    print(f"\nTesting LR = {lr}")
    
    # 1. CRÉER le modèle
    model = DeepHedgingNetwork(config['model'])
    
    # 2. SAUVEGARDER θ₀ IMMÉDIATEMENT (AVANT training!)
    init_path = f'../experiments/pruning/lr_{lr}/init_weights.pt'
    Path(init_path).parent.mkdir(parents=True, exist_ok=True)
    torch.save(model.state_dict(), init_path)  # θ₀ sauvegardé!
    print(f"  Initial weights (θ₀) saved to {init_path}")
    
    # 3. Configurer et entraîner le modèle dense
    config_temp = deepcopy(config)
    config_temp['training']['learning_rate'] = lr
    config_temp['training']['epochs'] = 100
    
    trainer = Trainer(model, config_temp, device, mask=None)
    trainer.fit(train_loader, val_loader, K, T, dt)
    print(f"  Dense training complete")
    
    # 4. Pruner à 80%
    mask = magnitude_pruning(model, sparsity=0.8)
    print(f"  Sparsity after pruning: {get_sparsity(model):.2%}")
    
    # 5. Créer un NOUVEAU modèle et rewind vers θ₀
    model_pruned = DeepHedgingNetwork(config['model'])
    rewind_weights(model_pruned, init_path, mask)  # Rewind vers θ₀!
    print(f"  Model rewound to θ₀ with mask applied")
    
    # 6. Réentraîner le modèle sparse avec mask
    config_temp['training']['epochs'] = 40
    trainer_retrain = Trainer(model_pruned, config_temp, device, mask=mask)
    trainer_retrain.fit(train_loader, val_loader, K, T, dt)
    
    # 7. Mesurer convergence
    epochs_to_95pct = sum(1 for loss in trainer_retrain.val_losses 
                          if loss > 0.95 * baseline_cvar)
    
    results_lr[lr] = {
        'epochs_to_95pct': epochs_to_95pct,
        'final_cvar': trainer_retrain.best_val_loss,
        'convergence_curve': trainer_retrain.val_losses
    }
    
    print(f"  Epochs to 95%: {epochs_to_95pct}")
    print(f"  Final CVaR: {trainer_retrain.best_val_loss:.6f}")

# Identifier le meilleur LR
best_lr = min(results_lr, key=lambda lr: results_lr[lr]['epochs_to_95pct'])
print(f"\nBest LR for boosting: {best_lr}")

# Sauvegarder résultats
with open('../experiments/pruning/lr_search_results.json', 'w') as f:
    json.dump({str(k): v for k, v in results_lr.items()}, f, indent=2)

### Experiment 2.2: Sparsity Ablation

In [ ]:
sparsities = config['pruning']['sparsities']
results_sparsity = {}

for sparsity in sparsities:
    print(f"\nTesting sparsity = {sparsity}")
    
    # 1. CRÉER le modèle
    model = DeepHedgingNetwork(config['model'])
    
    # 2. SAUVEGARDER θ₀ IMMÉDIATEMENT (AVANT training!)
    init_path = f'../experiments/pruning/sparsity_{int(sparsity*100)}/init_weights.pt'
    Path(init_path).parent.mkdir(parents=True, exist_ok=True)
    torch.save(model.state_dict(), init_path)  
    print(f"  Initial weights (θ₀) saved to {init_path}")
    
    # 3. Configurer et entraîner avec best_lr
    config_temp = deepcopy(config)
    config_temp['training']['learning_rate'] = best_lr
    config_temp['training']['epochs'] = 100
    
    trainer = Trainer(model, config_temp, device, mask=None)
    trainer.fit(train_loader, val_loader, K, T, dt)
    print(f"  Dense training complete")
    
    # 4. Pruner au niveau de sparsity
    mask = magnitude_pruning(model, sparsity=sparsity)
    print(f"  Sparsity: {get_sparsity(model):.2%}")
    
    # 5. Créer un NOUVEAU modèle et rewind vers θ₀
    model_pruned = DeepHedgingNetwork(config['model'])
    rewind_weights(model_pruned, init_path, mask)  # Rewind vers θ₀!
    print(f"  Model rewound to θ₀ with mask applied")
    
    # 6. Réentraîner le modèle sparse avec mask
    config_temp['training']['epochs'] = 40
    trainer_retrain = Trainer(model_pruned, config_temp, device, mask=mask)
    trainer_retrain.fit(train_loader, val_loader, K, T, dt)
    
    # 7. Évaluer
    metrics = compute_all_metrics(model_pruned, test_loader, config, K, T, dt, device)
    
    results_sparsity[sparsity] = {
        'cvar_005': metrics['cvar_005'],
        'mean_pnl': metrics['mean_pnl'],
        'sharpe_ratio': metrics['sharpe_ratio'],
        'final_val_loss': trainer_retrain.best_val_loss
    }
    
    print(f"  CVaR 0.05: {metrics['cvar_005']:.6f}")
    print(f"  Mean P&L: {metrics['mean_pnl']:.6f}")

# Sauvegarder résultats
with open('../experiments/pruning/sparsity_ablation_results.json', 'w') as f:
    json.dump({str(k): v for k, v in results_sparsity.items()}, f, indent=2)

## Visualization

In [ ]:
# Plot performance vs sparsity
sparsity_list = list(results_sparsity.keys())
cvar_list = [results_sparsity[s]['final_cvar'] for s in sparsity_list]

plot_sparsity_performance(
    sparsity_list,
    cvar_list,
    baseline_cvar,
    title="Performance vs Sparsity",
    save_path='../figures/performance_vs_sparsity.pdf'
)

## Summary

Best boosting ticket found at:
- LR: {best_lr}
- Sparsity: 80%
- Convergence: 2-3x faster than standard training